# **情報量規準**
重回帰分析や判別分析を実行する際に変数選択手法の特徴を理解し、適用できる

実際上、限られた個数の観測データから新の現象構造を正確に捉えるのは難しい。  
そこで、データをもとに多数の候補となる統計モデルを構築し、もっともよく当てはまっているモデルを選択する。

そのモデルの当てはまりの良さを評価する方法の一つに情報量規準がある。ここではその中でも特に有名な **AIC (赤池情報量規準)** と **BIC (ベイズ情報量規準)** について触れる。

### **カルバック・ライブラー情報量 (KL 情報量)**

AIC や BIC に触れる前に、理論的前提となる **カルバック・ライブラー情報量** について触れる。

カルバック・ライブラー情報量とは、2つの確率分布の差異をはかる尺度である。応用上は「真の」確率分布 $P$ とそれ以外の任意の確率分布 $Q$ に対するカルバック・ライブラー情報量が計算されることが多い。

カルバック・ライブラー情報量は以下の式で計算される。

- $ D_{KL}(P||Q) = \sum_i P(i) log \frac{P(i)}{Q(i)} $ ($P,Q$が離散確率分布の時。$P(i),Q(i)$は確率分布$P,Q$に従って選ばれた値が$i$の時の確率)
- $ D_{KL}(P||Q) = \int^∞_{-∞} p(x) log \frac{p(x)}{q(x)} $ ($P,Q$が連続確率分布の時。$p,q$は$P,Q$の確率密度関数)


### **AIC (赤池情報量規準)**

AIC は統計モデルを確率分布で表現し、その評価にカルバック・ライブラー情報量を採用することによって導かれたものである。

$ AIC = -2 log(L) + 2k $ として定義される。なお、ここで $L$ は最大尤度、$k$は自由パラメータの数である。

先述した通り、カルバック・ライブラー情報量は応用上は「真の」確率分布とそれ以外の任意の確率分布の差異の尺度として用いられることが多い。しかし、「真の」確率分布が事前に分かっているケースは稀なため、それを推定する必要がある。

カルバック・ライブラー情報量を推定したものとして、**対数尤度**が用いられる。しかし、これはカルバック・ライブラー情報量の不変推定量ではなく、バイアスをもった推定量であることが知られている。このバイアスが $2k$ のため、AIC は右辺の第二項にその値を含めている。

AIC の理論に従えば、AIC が最小になるモデルが良いモデルである。

### **BIC (ベイズ情報量規準)**

BIC も AIC と並んでよく用いられる情報量規準の一つである。

$ BIC = -2 log(L) + k log(n) $ として定義される。なお、ここで $L$ は最大尤度、$n$ は標本の大きさ、$k$ は自由パラメータの数である。

BIC の理論に従えば、BIC が最小になるモデルが良いモデルである。

### **AIC や BIC を用いたモデル選定**

statsmodels.api のライブラリを使えば、AIC や BIC も簡単に算出可能である。

In [37]:
# モジュールとデータの読み込み
import numpy as np
import statsmodels.api as sm
import scipy

spector_data = sm.datasets.spector.load()
spector_data.exog = sm.add_constant(spector_data.exog, prepend=False)

In [38]:
# OLS モデルの当てはめとサマリ
mod = sm.OLS(spector_data.endog, spector_data.exog)
res = mod.fit()

# サマリの　AIC, BIC の項にそれぞれ当該モデルの AIC, BIC の値が表示される
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                  GRADE   R-squared:                       0.416
Model:                            OLS   Adj. R-squared:                  0.353
Method:                 Least Squares   F-statistic:                     6.646
Date:                Mon, 14 Aug 2023   Prob (F-statistic):            0.00157
Time:                        13:21:30   Log-Likelihood:                -12.978
No. Observations:                  32   AIC:                             33.96
Df Residuals:                      28   BIC:                             39.82
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
GPA            0.4639      0.162      2.864      0.0

説明変数として含めるものを変えて AIC, BIC を算出する。

In [59]:
# GPA と切片のみのモデル
res = sm.OLS(spector_data.endog, spector_data.exog[["GPA", "const"]]).fit()

print("GPA と切片のみのモデル")
print("AIC: {}".format(res.aic))
print("BIC: {}".format(res.bic))

GPA と切片のみのモデル
AIC: 38.07764138094392
BIC: 41.009113186543374


In [60]:
# TUCE と切片のみのモデル
res = sm.OLS(spector_data.endog, spector_data.exog[["TUCE", "const"]]).fit()

print("TUCE と切片のみのモデル")
print("AIC: {}".format(res.aic))
print("BIC: {}".format(res.bic))

TUCE と切片のみのモデル
AIC: 44.07953467447594
BIC: 47.01100648007539


In [61]:
# PSI と切片のみのモデル
res = sm.OLS(spector_data.endog, spector_data.exog[["PSI", "const"]]).fit()

print("PSI と切片のみのモデル")
print("AIC: {}".format(res.aic))
print("BIC: {}".format(res.bic))

PSI と切片のみのモデル
AIC: 40.86158213731066
BIC: 43.79305394291011


In [62]:
# GPA, TUCE と切片のみのモデル
res = sm.OLS(spector_data.endog, spector_data.exog[["GPA", "TUCE", "const"]]).fit()

print("GPA, TUCE と切片のみのモデル")
print("AIC: {}".format(res.aic))
print("BIC: {}".format(res.bic))

GPA, TUCE と切片のみのモデル
AIC: 39.45945421565483
BIC: 43.85666192405401


In [63]:
# GPA, PSI と切片のみのモデル
res = sm.OLS(spector_data.endog, spector_data.exog[["GPA", "PSI", "const"]]).fit()

print("GPA, PSI と切片のみのモデル")
print("AIC: {}".format(res.aic))
print("BIC: {}".format(res.bic))

GPA, PSI と切片のみのモデル
AIC: 32.286421750768724
BIC: 36.6836294591679


In [64]:
# GPA, TUCE, PSI と切片全てを含むモデル
res = sm.OLS(spector_data.endog, spector_data.exog[["GPA", "TUCE", "PSI", "const"]]).fit()

print("GPA, TUCE, PSI と切片全てを含むモデル")
print("AIC: {}".format(res.aic))
print("BIC: {}".format(res.bic))

GPA, TUCE, PSI と切片全てを含むモデル
AIC: 33.95649234217083
BIC: 39.81943595336974


AIC, BIC ともに最小となるモデルは GPA, PSI と切片を含むモデルである。  
AIC, BIC いずれにおいても、最良のモデルは**GPA, PSI と切片を含むモデル**と言える。

### **参考文献**

- 赤池弘次「AIC と MDL と BIC」『オペレーションズ・リサーチ』41 (7)、1996年、375-378頁
- 黒住英司「情報量規準とその落とし穴」https://gcoe.ier.hit-u.ac.jp/vox/017.html、2023年8月アクセス
- 小西貞則「統計的モデリングと情報量規準」『応用数理』10 (3)、2000年、198-217頁